In [1]:
!pip install google-api-python-client google-auth google-auth-oauthlib markdown

In [2]:
from google.colab import auth
import google
auth.authenticate_user()
import re
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

SCOPES = ["https://www.googleapis.com/auth/documents"]
creds, _ = google.auth.default(scopes=SCOPES)
docs_service = build("docs", "v1", credentials=creds)


In [3]:
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [4]:
# Markdown to Google Doc Converter Logic

class MarkdownToGoogleDoc:
    def __init__(self):
        self.service = build('docs', 'v1')
        self.requests = []
        self.current_index = 1  # Docs start at index 1

    def create_doc(self, title):
        """Creates a new Google Doc and returns the ID."""
        try:
            doc = self.service.documents().create(body={'title': title}).execute()
            print(f"Created document: {doc.get('title')} (ID: {doc.get('documentId')})")
            return doc.get('documentId')
        except HttpError as err:
            print(f"An error occurred: {err}")
            return None

    def _add_text(self, text, style=None):
        """Helper to add insert text requests."""
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })

        end_index = self.current_index + len(text) + 1

        # Check for @mentions styling within the text
        mention_pattern = r'(@\w+)'
        for match in re.finditer(mention_pattern, text):
            start_mention = self.current_index + match.start()
            end_mention = self.current_index + match.end()
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': start_mention,
                        'endIndex': end_mention
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {'color': {'rgbColor': {'blue': 0.8, 'red': 0.0, 'green': 0.0}}}
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        # Apply Paragraph Styling (Headings)
        if style:
            self.requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': self.current_index,
                        'endIndex': end_index
                    },
                    'paragraphStyle': {'namedStyleType': style},
                    'fields': 'namedStyleType'
                }
            })

        return end_index

    def _process_list_item(self, text, nesting_level, is_checkbox=False):
        """Handles bullet points and checkboxes."""
        clean_text = re.sub(r'^(\s*)([-*]|\d+\.)(\s\[ \])?\s', '', text)

        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': clean_text + '\n'
            }
        })

        end_index = self.current_index + len(clean_text) + 1

        # Apply Bullet or Checkbox style
        bullet_preset = 'BULLET_CHECKBOX' if is_checkbox else 'BULLET_DISC_CIRCLE_SQUARE'

        self.requests.append({
            'createParagraphBullets': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index - 1 # Exclude newline
                },
                'bulletPreset': bullet_preset
            }
        })


        # Check for mentions
        mention_pattern = r'(@\w+)'
        for match in re.finditer(mention_pattern, clean_text):
            start_mention = self.current_index + match.start()
            end_mention = self.current_index + match.end()
            self.requests.append({
                'updateTextStyle': {
                    'range': {
                        'startIndex': start_mention,
                        'endIndex': end_mention
                    },
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {'color': {'rgbColor': {'blue': 0.8, 'red': 0.2, 'green': 0.2}}}
                    },
                    'fields': 'bold,foregroundColor'
                }
            })

        return end_index

    def _process_footer(self, text):
        """Styles the footer content."""
        self.requests.append({
            'insertText': {
                'location': {'index': self.current_index},
                'text': text + '\n'
            }
        })
        end_index = self.current_index + len(text) + 1

        self.requests.append({
            'updateTextStyle': {
                'range': {
                    'startIndex': self.current_index,
                    'endIndex': end_index
                },
                'textStyle': {
                    'italic': True,
                    'foregroundColor': {'color': {'rgbColor': {'red': 0.5, 'green': 0.5, 'blue': 0.5}}}
                },
                'fields': 'italic,foregroundColor'
            }
        })
        return end_index

    def parse_markdown(self, markdown_text, doc_id):
        """Main parsing loop."""
        lines = markdown_text.split('\n')

        is_footer = False

        for line in lines:
            if not line.strip():
                continue # Skip empty lines

            # Detect Footer Start
            if line.startswith('---') or line.startswith('Meeting recorded by'):
                is_footer = True
                if line.startswith('---'): continue

            # --- PARSING LOGIC ---

            if is_footer:
                self.current_index = self._process_footer(line)

            # Heading 1
            elif line.startswith('# '):
                self.current_index = self._add_text(line[2:], 'HEADING_1')

            # Heading 2
            elif line.startswith('## '):
                self.current_index = self._add_text(line[3:], 'HEADING_2')

            # Heading 3
            elif line.startswith('### '):
                self.current_index = self._add_text(line[4:], 'HEADING_3')

            # Checkboxes
            elif '- [ ]' in line:
                self.current_index = self._process_list_item(line, 0, is_checkbox=True)

            # Bullet points (simple nesting detection by whitespace)
            elif line.strip().startswith('*') or line.strip().startswith('-'):
                indent = len(line) - len(line.lstrip())
                level = indent // 2
                self.current_index = self._process_list_item(line, level, is_checkbox=False)

            # Standard Text
            else:
                self.current_index = self._add_text(line, 'NORMAL_TEXT')

        # Execute Batch Request
        if self.requests:
            try:
                self.service.documents().batchUpdate(
                    documentId=doc_id,
                    body={'requests': self.requests}
                ).execute()
                print("Formatting applied successfully.")
            except HttpError as err:
                print(f"Error applying formatting: {err}")

In [5]:

converter = MarkdownToGoogleDoc()

# Create Document
doc_id = converter.create_doc("Product Team Sync - Converted")

if doc_id:
    converter.parse_markdown(markdown_content, doc_id)

    print(f"\nSuccess! Open your document here: https://docs.google.com/document/d/{doc_id}")

Created document: Product Team Sync - Converted (ID: 1GYgovmWhf2sBVK8Bqv1HhOycupvrOKi0elBD1Id8COM)
Formatting applied successfully.

Success! Open your document here: https://docs.google.com/document/d/1GYgovmWhf2sBVK8Bqv1HhOycupvrOKi0elBD1Id8COM
